# Debug notebook

In [1]:
%matplotlib inline
import argparse
import os
import random
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.backends.cudnn as cudnn
import torch.optim as optim
import torchvision
from torchvision import models
import torch.utils.data as tud
import torchvision.datasets as dset
import torchvision.transforms as transforms
import torchvision.transforms.functional as TF
import torchvision.utils as vutils
from torch.utils.data import DataLoader, Dataset
from torch.utils.data.dataset import random_split
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from IPython.display import HTML
from tqdm import tqdm
from PIL import Image
from collections import Counter
from sklearn.metrics import jaccard_score
import pickle
import sys
sys.path.insert(1, '../utils')
sys.path.insert(1, '../datasets')
sys.path.insert(1, '../utils_on_gpu')
import coco_utils as cu
import my_datasets as mdset
import eval_train as ev
import utils as U
import find_best_model as fbm   




## Parameters

In [4]:
#MODEL SAVE AND LOAD 
dataroot_voc = '/share/DEEPLEARNING/datasets/voc2012'
dataroot_sbd = '/share/DEEPLEARNING/datasets/sbd'
dataroot_coco = '/share/DEEPLEARNING/datasets/coco'
dataroot_coco2voc = '/users/k/karmimy/data/coco2voc'
load_dir = '/share/homes/karmimy/equiv/save_model/fully_supervised' #load a particular model 
fcn= True
pretrained=True

# GPU 
gpu = 3
# TRAIN PARAMETERS
batch_size = 2
gamma = 0.5
learning_rate = 10e-4
moment = 0.9
wd = 2e-4


# LOSS 

criterion_supervised = nn.CrossEntropyLoss(ignore_index=21) # On ignore la classe border.
Loss = 'KL' # Loss = 'KL' or 'CE' or None for L1,MSE…
criterion_unsupervised = U.get_criterion(Loss)


# DATASET AND DATA AUG 
rotate = False # random rotation during training
scale = True
split = True # split the supervised dataset
ratio = 0.3 # percent of data of VOC + SBD we use for supervised dataset
#scale_factor = (0.2,0.8)
#size_img = (420,420) 
#size_crop = (380,380)

## DATASETS

In [5]:
train_dataset_VOC = mdset.VOCSegmentation(dataroot_voc,year='2012', image_set='train', \
        download=True,rotate=rotate,scale=scale)#,size_img=size_img,size_crop=size_crop)
val_dataset_VOC = mdset.VOCSegmentation(dataroot_voc,year='2012', image_set='val', download=True)
train_dataset_SBD = mdset.SBDataset(dataroot_sbd, image_set='train_noval',mode='segmentation',\
        rotate=rotate,scale=scale)#,size_img=size_img,size_crop=size_crop)

Using downloaded and verified file: /share/DEEPLEARNING/datasets/voc2012/VOCtrainval_11-May-2012.tar
Using downloaded and verified file: /share/DEEPLEARNING/datasets/voc2012/VOCtrainval_11-May-2012.tar


In [6]:
i,m=train_dataset_VOC.__getitem__(0)

In [7]:
torch.unique(m)

tensor([ 0,  1, 15])

In [9]:


dataloader_train_VOC = torch.utils.data.DataLoader(train_dataset_VOC, batch_size=batch_size,\
                                                       shuffle=True,drop_last=True)
#dataloader_train_sup = torch.utils.data.DataLoader(train_dataset_sup, batch_size=batch_size,\
#                                                       shuffle=True,drop_last=True)
#dataloader_train_equiv = torch.utils.data.DataLoader(train_dataset_unsup,batch_size=batch_size,\
#                                                     shuffle=True,drop_last=True)

dataloader_val = torch.utils.data.DataLoader(val_dataset_VOC, batch_size=batch_size)
# Decide which device we want to run on
device = torch.device("cuda:"+str(gpu) if torch.cuda.is_available() else "cpu")
print("device :",device)



device : cuda:3


In [8]:
print("Taille dataset train supervised :",len(train_dataset_sup))
print("Taille dataset train unsupervised :",len(train_dataset_unsup))
print("Taille dataset val VOC :",len(val_dataset_VOC))

Taille dataset train supervised : 2126
Taille dataset train unsupervised : 7087
Taille dataset val VOC : 1449



## FCN Pytorch

In [9]:
def load_model(file=None,fcn=False,pretrained=False):
    if file is None:
        if fcn is False:
            model = torchvision.models.segmentation.deeplabv3_resnet101(pretrained=pretrained)
        else:
            model = torchvision.models.segmentation.fcn_resnet101(pretrained=pretrained)
    else:
        model = torch.load(os.path.join(SAVE_DIR,file))
    return model

In [10]:
model = load_model(file=None,fcn=fcn,pretrained=False)

In [11]:
model.to(device)

)
    )
    (layer2): Sequential(
      (0): Bottleneck(
        (conv1): Conv2d(256, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv2d(128, 128, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv3): Conv2d(128, 512, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn3): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (downsample): Sequential(
          (0): Conv2d(256, 512, kernel_size=(1, 1), stride=(2, 2), bias=False)
          (1): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        )
      )
      (1): Bottleneck(
        (conv1): Conv2d(512, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn1): BatchNorm2d(128

## Test Loss


In [12]:
optimizer = torch.optim.SGD(model.parameters(),lr=learning_rate,momentum=moment, weight_decay=wd)

In [13]:
batch = next(iter(dataloader_train_equiv))

In [14]:
x,m = batch

In [15]:
min_f,max_f = scale_factor
factor = random.uniform(min_f,max_f) 
size = int(size_img[0] * factor)
print(size)
print(criterion_unsupervised)

142
CrossEntropyLoss()


In [16]:
loss_equiv,acc = U.compute_scale_equiv_batch(x,model,size=(size,size),\
                                                     criterion=criterion_unsupervised,Loss = Loss,\
                                                       device=device)

In [17]:
loss_equiv

tensor(2.6292, grad_fn=<NllLoss2DBackward>)

In [24]:
loss_sup = criterion_supervised(x,m)

IndexError: Target 17 is out of bounds.

In [30]:
for batch_sup,batch_unsup in zip(dataloader_train_sup,dataloader_train_equiv):
    optimizer.zero_grad()
    min_f,max_f = scale_factor
    factor = random.uniform(min_f,max_f) 
    size = int(size_img[0] * factor)
    x_unsup,_ = batch_unsup
    loss_equiv,acc = U.compute_scale_equiv_batch(x_unsup,model,size=(size,size),\
                                                criterion=criterion_unsupervised,Loss = Loss,\
                                                device=device)
    print('loss equiv',loss_equiv)
    x,mask = batch_sup
    x = x.to(device)
    mask = mask.to(device)
    pred = model(x)["out"]
    loss_equiv = loss_equiv.to(device) # otherwise bug in combining the loss 
    loss_sup = criterion_supervised(pred,mask)
    print('loss_sup',loss_sup)
    loss = gamma*loss_sup + (1-gamma)*loss_equiv # combine loss 
    print('loss',loss)             
    loss.backward()
    optimizer.step()

uda:1', grad_fn=<NllLoss2DBackward>)
loss tensor(0.7025, device='cuda:1', grad_fn=<AddBackward0>)
loss equiv tensor(0.2171, grad_fn=<NllLoss2DBackward>)
loss_sup tensor(0.7064, device='cuda:1', grad_fn=<NllLoss2DBackward>)
loss tensor(0.4618, device='cuda:1', grad_fn=<AddBackward0>)
loss equiv tensor(0.2515, grad_fn=<NllLoss2DBackward>)
loss_sup tensor(1.9837, device='cuda:1', grad_fn=<NllLoss2DBackward>)
loss tensor(1.1176, device='cuda:1', grad_fn=<AddBackward0>)
loss equiv tensor(0.3035, grad_fn=<NllLoss2DBackward>)
loss_sup tensor(0.7732, device='cuda:1', grad_fn=<NllLoss2DBackward>)
loss tensor(0.5384, device='cuda:1', grad_fn=<AddBackward0>)
loss equiv tensor(0.3183, grad_fn=<NllLoss2DBackward>)
loss_sup tensor(0.7927, device='cuda:1', grad_fn=<NllLoss2DBackward>)
loss tensor(0.5555, device='cuda:1', grad_fn=<AddBackward0>)
loss equiv tensor(0.4958, grad_fn=<NllLoss2DBackward>)
loss_sup tensor(0.5846, device='cuda:1', grad_fn=<NllLoss2DBackward>)
loss tensor(0.5402, device='cuda:

## Plot